In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [3]:
# Column Names (['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_content', 'img_link', 'product_link'],  dtype='object')
fullDataset = pd.read_csv("C:\\Repos\\MastersProject\\data-analysis\\Data\\amazon.csv", delimiter=",")

fullUserData = pd.read_csv("C:\\Repos\\MastersProject\\data-analysis\\Data\\userInfo.csv", delimiter=",")

# Part - I - content based filtering using Count Vectorizer

In [4]:
# ignore the columns that are not needed
products_dataset = fullDataset [
    [
        "product_id",
        "product_name",
        "category",
        "actual_price",
        "img_link",
        "product_link",
        "about_product",
        "rating",
        "rating_count"   
    ]
]
        
products_dataset.rename(columns={
    "actual_price": 'price',
    "about_product":'product_description',
    "rating_count":'no_of_ratings'
},inplace=True)


#df['Salary'].str.replace(',', '').str.replace('₹', '').astype(int)
products_dataset['no_of_ratings'] = products_dataset['no_of_ratings'].str.replace('[,]','', regex=True).astype(int)        
products_dataset['rating'] = (products_dataset['rating'].astype(float)).round(1)
products_dataset['price'] = (products_dataset['price'].str.replace('[\₹\,]','', regex=True).astype(float) / 80).round(2)
products_dataset["category"] = products_dataset["category"].str.split("|") 

products_dataset.drop_duplicates(subset=['product_id'], inplace=True)


C:\Users\srile\AppData\Local\Temp\ipykernel_13316\3840223352.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_dataset.rename(columns={
C:\Users\srile\AppData\Local\Temp\ipykernel_13316\3840223352.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_dataset['no_of_ratings'] = products_dataset['no_of_ratings'].str.replace('[,]','', regex=True).astype(int)
C:\Users\srile\AppData\Local\Temp\ipykernel_13316\3840223352.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [7]:
products_dataset['product_name'] =products_dataset['product_name'].str.lower() 
products_dataset['product_description'] =products_dataset['product_description'].str.lower() 
products_dataset.head()

C:\Users\srile\AppData\Local\Temp\ipykernel_13316\3777469439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_dataset['product_name'] =products_dataset['product_name'].str.lower()
C:\Users\srile\AppData\Local\Temp\ipykernel_13316\3777469439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_dataset['product_description'] =products_dataset['product_description'].str.lower()


,product_id,product_name,category,price,img_link,product_link,product_description,rating,no_of_ratings
0,B07JW9H4J1,wayona nylon braided usb to lightning fast cha...,"[Computers&Accessories, Accessories&Peripheral...",13.74,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,high compatibility : compatible with iphone 12...,4.2,24269
1,B098NS6PVG,ambrane unbreakable 60w / 3a fast charging 1.5...,"[Computers&Accessories, Accessories&Peripheral...",4.36,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...,"compatible with all type c enabled devices, be...",4.0,43994
2,B096MSW6CT,sounce fast phone charging cable & data sync u...,"[Computers&Accessories, Accessories&Peripheral...",23.74,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...,【 fast charger& data sync】-with built-in safet...,3.9,7928
3,B08HDJ86NZ,boat deuce usb 300 2 in 1 type-c & micro usb s...,"[Computers&Accessories, Accessories&Peripheral...",8.74,https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...,the boat deuce usb 300 2 in 1 cable is compati...,4.2,94363
4,B08CF3B7N1,portronics konnect l 1.2m fast charging 3a 8 p...,"[Computers&Accessories, Accessories&Peripheral...",4.99,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...,[charge & sync function]- this cable comes wit...,4.2,16905


In [16]:
new_subset_products_dataset = products_dataset.drop(['product_id','category','price','img_link','product_link','rating','no_of_ratings'], axis=1)

new_subset_products_dataset['data'] = new_subset_products_dataset[new_subset_products_dataset.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

new_subset_products_dataset['data']

0       high compatibility : compatible with iphone 12...
1       compatible with all type c enabled devices, be...
2       【 fast charger& data sync】-with built-in safet...
3       the boat deuce usb 300 2 in 1 cable is compati...
4       [charge & sync function]- this cable comes wit...
                              ...                        
1460    supreme quality 90 gram 3 layer thik pp spun f...
1461                         230 volts, 400 watts, 1 year
1462    international design and styling|two heat sett...
1463    fan sweep area: 230 mm ; noise level: (40 - 45...
1464    brand-borosil, specification â€“ 23v ~ 5hz;1 w...
Name: data, Length: 1351, dtype: object

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(new_subset_products_dataset['data'])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

print(similarities)

[[1.         0.26652486 0.47174595 ... 0.17762742 0.16994262 0.10533126]
 [0.26652486 1.         0.212468   ... 0.05709161 0.07458677 0.02979218]
 [0.47174595 0.212468   1.         ... 0.32169134 0.21974926 0.21395011]
 ...
 [0.17762742 0.05709161 0.32169134 ... 1.         0.24603392 0.13266892]
 [0.16994262 0.07458677 0.21974926 ... 0.24603392 1.         0.16048518]
 [0.10533126 0.02979218 0.21395011 ... 0.13266892 0.16048518 1.        ]]


In [19]:
products_dataset = pd.DataFrame(similarities, columns=products_dataset['product_name'], index=products_dataset['product_name']).reset_index()

products_dataset.head()

product_name,product_name,"wayona nylon braided usb to lightning fast charging and data sync cable compatible for iphone 13, 12,11, x, 8, 7, 6, 5, ipad air, pro, mini (3 ft pack of 1, grey)","ambrane unbreakable 60w / 3a fast charging 1.5m braided type c cable for smartphones, tablets, laptops & other type c devices, pd technology, 480mbps data sync, quick charge 3.0 (rct15a, black)","sounce fast phone charging cable & data sync usb cable compatible for iphone 13, 12,11, x, 8, 7, 6, 5, ipad air, pro, mini & ios devices","boat deuce usb 300 2 in 1 type-c & micro usb stress resistant, tangle-free, sturdy cable with 3a fast charging & 480mbps data transmission, 10000+ bends lifespan and extended 1.5m length(martian red)","portronics konnect l 1.2m fast charging 3a 8 pin usb cable with charge & sync function for iphone, ipad (grey)","ptron solero tb301 3a type-c data and fast charging cable, made in india, 480mbps data sync, strong and durable 1.5-meter nylon braided usb cable for type-c devices for charging adapter (black)","boat micro usb 55 tangle-free, sturdy micro usb cable with 3a fast charging & 480mbps data transmission (black)",mi usb type-c cable smartphone (black),"tp-link usb wifi adapter for pc(tl-wn725n), n150 wireless network adapter for desktop - nano size wifi dongle compatible with windows 11/10/7/8/8.1/xp/ mac os 10.9-10.15 linux kernel 2.6.18-4.4.3",...,"usha 1212 ptc with adjustable thermostat fan heater (black/brown, 1500-watts).","4 in 1 handheld electric vegetable cutter set,wireless food processor electric food chopper for garlic chili pepper onion ginger celery meat with brush",philips hd9306/06 1.5-litre electric kettle (multicolor),"libra room heater for home, room heaters home for winter, electric heater with 2000 watts power as per is specification for small to medium rooms - fh12 (grey)",ngi store 2 pieces pet hair removers for your laundry catcher lint remover for washing machine lint remover reusable portable silica gel clothes washer dryer floating ball,"noir aqua - 5pcs pp spun filter + 1 spanner | for all types of ro water purifiers (5 piece, white, 10 inch, 5 micron) - ro spun filter cartridge sponge replacement water filter candle","prestige delight prwo electric rice cooker (1 l, white)","bajaj majesty rx10 2000 watts heat convector room heater (white, isi approved)",havells ventil air dsp 230mm exhaust fan (pista green),borosil jumbo 1000-watt grill sandwich maker (black)
0,wayona nylon braided usb to lightning fast cha...,1.000000,0.266525,0.471746,0.360209,0.467516,0.303264,0.380305,0.238705,0.197950,...,0.038742,0.378143,0.171718,0.185815,0.329480,0.157739,0.0,0.177627,0.169943,0.105331
1,ambrane unbreakable 60w / 3a fast charging 1.5...,0.266525,1.000000,0.212468,0.367551,0.357570,0.352875,0.393254,0.346019,0.136558,...,0.013697,0.295976,0.063351,0.188209,0.212592,0.169231,0.0,0.057092,0.074587,0.029792
2,sounce fast phone charging cable & data sync u...,0.471746,0.212468,1.000000,0.423893,0.454686,0.431854,0.469622,0.349659,0.324378,...,0.037833,0.570399,0.285803,0.249135,0.521239,0.288956,0.0,0.321691,0.219749,0.213950
3,boat deuce usb 300 2 in 1 type-c & micro usb s...,0.360209,0.367551,0.423893,1.000000,0.395885,0.427781,0.780506,0.410327,0.211775,...,0.103339,0.445468,0.168168,0.285773,0.368627,0.141862,0.0,0.258435,0.166731,0.141519
4,portronics konnect l 1.2m fast charging 3a 8 p...,0.467516,0.357570,0.454686,0.395885,1.000000,0.414243,0.415917,0.296077,0.209596,...,0.084094,0.533926,0.194469,0.351941,0.434212,0.209145,0.0,0.180262,0.145371,0.191616


In [21]:
input_book = "wayona nylon braided usb to lightning fast charging and data sync cable compatible for iphone 13, 12,11, x, 8, 7, 6, 5, ipad air, pro, mini (3 ft pack of 1, grey)"

recommendations = pd.DataFrame(products_dataset.nlargest(11,input_book)['product_name'])
recommendations = recommendations[recommendations['product_name']!=input_book]
print(recommendations)

                                          product_name
220  wayona nylon braided usb syncing and charging ...
42   wayona nylon braided 3a lightning to usb a syn...
89   wayona nylon braided (2 pack) lightning fast u...
166  wayona nylon braided lightning usb data sync &...
80   wayona usb nylon braided data sync and chargin...
106  wayona nylon braided 2m / 6ft fast charge usb ...
208  myvn ltg to usb for fast charging & data sync ...
261  wayona nylon braided usb type c 3ft 1m 3a fast...
104  wayona nylon braided usb data sync and fast ch...
78   swapkart fast charging cable and data sync usb...


# Deep Auto Encoder and producing recommendations for a single user

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [24]:
Dataset = fullDataset[
    [
        "user_id",
        "product_id",
        "rating"
    ]
]
Dataset

,user_id,product_id,rating
0,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",B07JW9H4J1,4.2
1,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",B098NS6PVG,4.0
2,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...",B096MSW6CT,3.9
3,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...",B08HDJ86NZ,4.2
4,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...",B08CF3B7N1,4.2
...,...,...,...
1460,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...",B08L7J3T31,4.0
1461,"AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...",B01M6453MB,4.1
1462,"AGVPWCMAHYQWJOQKMUJN4DW3KM5Q,AF4Q3E66MY4SR7YQZ...",B009P2LIL4,3.6
1463,"AF2JQCLSCY3QJATWUNNHUSVUPNQQ,AFDMLUXC5LS5RXDJS...",B00J5DYCCA,4.0


In [83]:
New_Dataset = pd.DataFrame(Dataset)

# Split the 'user_id' column into multiple rows
New_Dataset = New_Dataset.assign(user_id=Dataset['user_id'].str.split(',')).explode('user_id')

New_Dataset = New_Dataset.drop_duplicates(['user_id', 'product_id'])
# Reset index
New_Dataset.reset_index(drop=True, inplace=True)

print(New_Dataset)
New_Dataset.to_csv('required_dataset.csv', index=False)

                            user_id  product_id  rating
0      AG3D6O4STAQKAY2UVGEUV46KN35Q  B07JW9H4J1     4.2
1      AHMY5CWJMMK5BJRBBSNLYT3ONILA  B07JW9H4J1     4.2
2      AHCTC6ULH4XB6YHDY6PCH2R772LQ  B07JW9H4J1     4.2
3      AGYHHIERNXKA6P5T7CZLXKVPT7IQ  B07JW9H4J1     4.2
4      AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q  B07JW9H4J1     4.2
...                             ...         ...     ...
10599  AHXCDNSXAESERITAFELQABFVNLCA  B01486F4G6     4.3
10600  AGRZD6CHLCUNOLMMIMIHUCG7PIFA  B01486F4G6     4.3
10601  AFQZVGSOSOJHKFQQMCEI4725QEKQ  B01486F4G6     4.3
10602  AEALVGXXIP46OZVXKRUXSDWZJMEA  B01486F4G6     4.3
10603  AGEFL3AY7YXEFZA4ZJU3LP7K7OJQ  B01486F4G6     4.3

[10604 rows x 3 columns]


In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Load the MovieLens dataset (download it from https://grouplens.org/datasets/movielens/)
file_path = 'C:\\Repos\\MastersProject\\data-analysis\\Others\\New\\required_dataset.csv'
columns = ['user_id', 'product_id', 'rating']
# df = pd.read_csv(file_path, sep='\t', names=columns)
df = pd.read_csv(file_path, sep=',')
df

,user_id,product_id,rating
0,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,4.2
1,AHMY5CWJMMK5BJRBBSNLYT3ONILA,B07JW9H4J1,4.2
2,AHCTC6ULH4XB6YHDY6PCH2R772LQ,B07JW9H4J1,4.2
3,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,B07JW9H4J1,4.2
4,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,B07JW9H4J1,4.2
...,...,...,...
10599,AHXCDNSXAESERITAFELQABFVNLCA,B01486F4G6,4.3
10600,AGRZD6CHLCUNOLMMIMIHUCG7PIFA,B01486F4G6,4.3
10601,AFQZVGSOSOJHKFQQMCEI4725QEKQ,B01486F4G6,4.3
10602,AEALVGXXIP46OZVXKRUXSDWZJMEA,B01486F4G6,4.3


In [85]:
# Create user-item interaction matrix
user_product_matrix = df.pivot(index='user_id', columns='product_id', values='rating').fillna(0).values

# Split the data into training and testing sets
train_data, test_data = train_test_split(user_product_matrix, test_size=0.2, random_state=42)

# Build the autoencoder model
num_users, num_items = user_product_matrix.shape
latent_dim = 50

input_layer = Input(shape=(num_items,))
encoded = Dense(latent_dim, activation='relu')(input_layer)
decoded = Dense(num_items, activation='sigmoid')(encoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(train_data, train_data, epochs=10, batch_size=64, shuffle=True, validation_data=(test_data, test_data))

# Extract user and item representations from the encoder part of the autoencoder
encoder = Model(inputs=input_layer, outputs=encoded)
user_embeddings = encoder.predict(user_product_matrix)

Epoch 1/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2180 - val_loss: 0.0554
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0363 - val_loss: 0.0186
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0172 - val_loss: 0.0159
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0156 - val_loss: 0.0152
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0148 - val_loss: 0.0149
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0150 - val_loss: 0.0148
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0147 - val_loss: 0.0147
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0148 - val_loss: 0.0147
Epoch 9/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0147 - val_loss: 0.0146
Epoch 10/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0145 - val_loss: 0.0146
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [86]:
# Example: Recommend items for a specific user
user_id = "AENGU523SXMOS7JPDTW52PNNVWGQ"  # Replace with the desired user ID
user_index = df.index[df['user_id'] == user_id].tolist()[0]
user_representation = user_embeddings[user_index - 1]

# Calculate the predicted ratings for all items
predicted_ratings = np.dot(user_embeddings, user_representation)

# Display top N recommendations
top_n = np.argsort(predicted_ratings)[::-1][:10]
print(f"Top recommendations for User {user_id}: {top_n + 1}")

Top recommendations for User AENGU523SXMOS7JPDTW52PNNVWGQ: [2668 2956 4777  228  610 3200 8054 4271 9001 4634]


In [87]:
for x in top_n:
    print(df.loc[x+1,"product_id"])

B0B3CQBRB4
B09XB7SRQ5
B08HLZ28QC
B09W5XR9RT
B09YLXYP7Y
B09MT6XSFW
B0BMFD94VD
B09NVPJ3P4
B0BCKJJN8R
B08H9Z3XQW


In [45]:
import numpy as np

# Assume user_embeddings is the matrix of user embeddings obtained after training the autoencoder
# Assume user_id is the desired user ID
user_id = "AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q"

# Step 1: Identify the index corresponding to the user ID
user_index = df.index[df['user_id'] == user_id].tolist()[0]

# Step 2: Retrieve the user representation from the encoded user-item interaction matrix
user_representation = user_embeddings[user_index]

print("User Representation:", user_representation)
user_index
#df

User Representation: [0.43162385 0.64218426 0.56667376 0.29643464 0.5109464  0.56116956
 0.57605183 0.63369274 0.37274116 0.5769812  0.7169067  0.5500021
 0.48876852 0.38465887 0.38756013 0.60319173 0.66712123 0.32377943
 0.332133   0.33549345 0.6563204  0.50890315 0.6810894  0.69304776
 0.5152472  0.62854636 0.42051935 0.5773324  0.23982605 0.47626266
 0.39427418 0.60837775 0.43895888 0.34954035 0.3873111  0.43745455
 0.33178455 0.4286843  0.         0.54803306 0.3229336  0.34250903
 0.328107   0.4402268  0.3533833  0.6042641  0.74704385 0.66780776
 0.46991515 0.7418378 ]


4

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Load the MovieLens dataset (download it from https://grouplens.org/datasets/movielens/)
file_path = 'C:\\Repos\\MastersProject\\data-analysis\\Data\\ratings.dat'
columns = ['user_id', 'item_id', 'rating']
df = pd.read_csv(file_path, sep='\t', names=columns)

# Create user-item interaction matrix
user_item_matrix = df.pivot(index='user_id', columns='item_id', values='rating').fillna(0).values

# Split the data into training and testing sets
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)

# Build the autoencoder model
num_users, num_items = user_item_matrix.shape
latent_dim = 50

input_layer = Input(shape=(num_items,))
encoded = Dense(latent_dim, activation='relu')(input_layer)
decoded = Dense(num_items, activation='sigmoid')(encoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(train_data, train_data, epochs=10, batch_size=64, shuffle=True, validation_data=(test_data, test_data))

# Extract user and item representations from the encoder part of the autoencoder
encoder = Model(inputs=input_layer, outputs=encoded)
user_embeddings = encoder.predict(user_item_matrix)

# Example: Recommend items for a specific user
user_id = 1  # Replace with the desired user ID
user_representation = user_embeddings[user_id - 1]

# Calculate the predicted ratings for all items
predicted_ratings = np.dot(user_embeddings, user_representation)

# Display top N recommendations
top_n = np.argsort(predicted_ratings)[::-1][:10]
print(f"Top recommendations for User {user_id}: {top_n + 1}")

Epoch 1/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 13s 995us/step - loss: 0.0630 - val_loss: 1.1202e-04
Epoch 2/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 11s 892us/step - loss: 4.4310e-05 - val_loss: 3.0128e-07
Epoch 3/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - loss: 1.2346e-07 - val_loss: 4.4438e-09
Epoch 4/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 11s 884us/step - loss: 3.1766e-09 - val_loss: 1.4309e-09
Epoch 5/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 11s 897us/step - loss: 1.2376e-09 - val_loss: 8.4173e-10
Epoch 6/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 12s 931us/step - loss: 7.6769e-10 - val_loss: 5.9492e-10
Epoch 7/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 11s 889us/step - loss: 5.5616e-10 - val_loss: 4.5937e-10
Epoch 8/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 12s 970us/step - loss: 4.3582e-10 - val_loss: 3.7452e-10
Epoch 9/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 11s 889us/step - loss: 3.5827e-10 - val_loss: 3.1538e-10
Epoch 10/10
12503/12503 ━━━━━━━━━━━━━━━━━━━━ 11s 883us/step - loss: 3.0405e-10 - val_loss: 2.7291